In [75]:
import pandas as pd
import numpy as np
from pytket.circuit import StatePreparationBox
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter as draw
from pytket.utils import QubitPauliOperator
from pytket.pauli import Pauli, QubitPauliString
import networkx as nx
from pytket.circuit import Qubit
from sympy import symbols
from pytket.extensions.qiskit import AerBackend, AerStateBackend
from pytket.utils.expectations import get_operator_expectation_value
from scipy.linalg import expm
import matplotlib.pyplot as plt
import plotly.express as px


In [27]:

population = pd.read_csv('population.csv')
covid = pd.read_csv('WHO-COVID-19-global-data.csv')

covid['Date_reported'] = pd.to_datetime(covid['Date_reported'])
df = covid.sort_values(by=['Country', 'Date_reported'])

df['Year'] = df['Date_reported'].dt.year
df['Month'] = df['Date_reported'].dt.month

last_week_of_month = df.groupby(['Country', 'Year', 'Month']).apply(lambda x: x.loc[x['Date_reported'].idxmax()])
last_week_of_month = last_week_of_month.reset_index(drop=True)

relevant_countries = ['China', 'United States of America', 'India', 'Russia', 'Argentina']
filtered_data = last_week_of_month[last_week_of_month['Country'].isin(relevant_countries)]


pivoted_data = filtered_data.pivot_table(
    index=['Year', 'Month'], 
    columns='Country',       
    values=['Cumulative_cases', 'Cumulative_deaths'] 
)

pivoted_data.columns = [f'{country}_{metric}' for metric, country in pivoted_data.columns]
pivoted_data = pivoted_data.reset_index()
print(pivoted_data)

pivoted_data.to_csv('transformed_covid_data.csv', index=False)

df=pivoted_data.drop(columns=['Year', 'Month'])
df

    Year  Month  Argentina_Cumulative_cases  China_Cumulative_cases  \
0   2020      1                         0.0                  1985.0   
1   2020      2                         0.0                 77042.0   
2   2020      3                      1282.0                 82341.0   
3   2020      4                      5464.0                 84338.0   
4   2020      5                     22406.0                 84570.0   
..   ...    ...                         ...                     ...   
56  2024      9                  10103850.0              99380642.0   
57  2024     10                  10106910.0              99381078.0   
58  2024     11                  10109740.0              99381255.0   
59  2024     12                  10110676.0              99381579.0   
60  2025      1                  10110920.0              99381761.0   

    India_Cumulative_cases  United States of America_Cumulative_cases  \
0                      0.0                                        7.0   
1

/var/folders/nt/nmz85frs1_z27cy8b2m7lt380000gn/T/ipykernel_32363/3912994157.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_week_of_month = df.groupby(['Country', 'Year', 'Month']).apply(lambda x: x.loc[x['Date_reported'].idxmax()])


,Argentina_Cumulative_cases,China_Cumulative_cases,India_Cumulative_cases,United States of America_Cumulative_cases,Argentina_Cumulative_deaths,China_Cumulative_deaths,India_Cumulative_deaths,United States of America_Cumulative_deaths
0,0.0,1985.0,0.0,7.0,0.0,56.0,0.0,0.0
1,0.0,77042.0,7.0,40.0,0.0,2446.0,0.0,0.0
2,1282.0,82341.0,979.0,126309.0,70.0,3306.0,25.0,2106.0
3,5464.0,84338.0,26496.0,922947.0,411.0,4642.0,824.0,53434.0
4,22406.0,84570.0,182143.0,1785422.0,1306.0,4645.0,5164.0,105325.0
...,...,...,...,...,...,...,...,...
56,10103850.0,99380642.0,45043605.0,103436829.0,130689.0,122360.0,533646.0,1203668.0
57,10106910.0,99381078.0,45044196.0,103436829.0,130702.0,122367.0,533653.0,1206802.0
58,10109740.0,99381255.0,45044456.0,103436829.0,130718.0,122377.0,533657.0,1208922.0
59,10110676.0,99381579.0,45044549.0,103436829.0,130736.0,122391.0,533661.0,1211450.0


In [58]:
def H_1_block(n_qubits: int, t: float, delta: float, J: float) -> Circuit:
    H_1_block = Circuit(n_qubits)

    for index in range(n_qubits-1):
        H_1_block.ZZPhase(2/np.pi*2*(delta*t), index, index +1)
        
    return H_1_block

def H_2_block(n_qubits: int, t: float, delta: float, h:float) -> Circuit:
    H_2_block = Circuit(n_qubits)

    for index in range(n_qubits):
        H_2_block.Rx(2/np.pi*2*(delta*t)*h, index)
        # H_2_block.Rx(1, index)

    return H_2_block

In [59]:
def prepare_first_order(n_qubits: int, t: float, delta: float, J: float, h:float) -> Circuit:
    first_order_trotter = Circuit(n_qubits)
    
    first_order_trotter.append(H_1_block(n_qubits,t,delta/2,J))
    first_order_trotter.append(H_2_block(n_qubits,t,delta/2,h))

    return first_order_trotter

def prepare_second_order(n_qubits: int, t: float, delta: float, J: float, h:float) -> Circuit:
    second_order_trotter = Circuit(n_qubits)
    
    second_order_trotter.append(H_1_block(n_qubits,t,2*delta,J))
    second_order_trotter.append(H_2_block(n_qubits,t,delta,h))
    second_order_trotter.append(H_1_block(n_qubits,t,2*delta,J))

    return second_order_trotter

In [60]:
def observable_Z(nqubits: int):
    magnetization_op = {}
    for i in range(nqubits):
        term = QubitPauliString({Qubit(i): Pauli.Z})
        magnetization_op[term] = 1.0 / nqubits
    magnetization_obs = QubitPauliOperator(magnetization_op)
    return magnetization_obs

def coeff(k: int):
    return (1/(4-4**(1/(2*k-1))))

In [61]:
def recursor(order:int,n_qubits: int, t: float, delta: float, h:float, J:float):
    if order==1:
        return prepare_first_order(n_qubits,t,delta,J,h)
    elif order%2:
        raise ValueError("cant do odd powers")
    elif order ==2: 
        return prepare_second_order(n_qubits, t, delta,J,h)
    else:
        S_recursor = Circuit(n_qubits)
        S1 = recursor(order-2,n_qubits,t,delta*coeff(order),h,J)
        S2 = recursor(order-2,n_qubits,t,(1-4*coeff(order))*delta,h,J)
        S_recursor.append(S1)
        S_recursor.append(S1)
        S_recursor.append(S2)
        S_recursor.append(S1)
        S_recursor.append(S1)
        return S_recursor

In [62]:
def time_evolve(n_qubits,t,delta, order,h,J):
    #create helpers
    sv_backend = AerStateBackend()
    t_arr = np.linspace(0,t,int(t/delta))
    
    T_S_Circuit = recursor(order=order, n_qubits=n_qubits, t=t, delta=delta,h=h,J=J)
    draw(T_S_Circuit)
    
    operator =observable_Z(n_qubits)
    
    mag_values = []
    initial_val = get_operator_expectation_value(T_S_Circuit,operator,sv_backend)
    mag_values.append(initial_val)
    
    
    sim_circ = recursor(order=order, n_qubits=n_qubits, t=t, delta=delta,h=h,J=J)
    for index in t_arr[1:]:
        T_S_Circuit.append(sim_circ)
        if i%10:
            mag_values.append(get_operator_expectation_value(T_S_Circuit,operator,sv_backend,shots=1000))
    return t_arr, mag_values

In [63]:
from pytket.extensions.qiskit import AerStateBackend

#initialize params
n_qubits = 3
t = 3
delta = 0.1

t_arr = np.linspace(0,t+delta,int(t/delta))

h=1.2
J=0.2

# h=3.
# J=1.


In [64]:

def get_ising_chain_hamiltonian(n_qubits: int, J: float, h:float) -> QubitPauliOperator:
    sites = nx.path_graph(n_qubits)
    qpo_dict = {}
    for e in sites.edges:
        zz_term = QubitPauliString([Qubit(e[0]), Qubit(e[1])], [Pauli.Z, Pauli.Z])
        qpo_dict[zz_term] = J
        x_term = QubitPauliString([Qubit(e[0])],[Pauli.X])
        qpo_dict[x_term] = h

    return QubitPauliOperator(qpo_dict)

def get_M(n_qubits,J,t):

    ham_qu = get_ising_chain_hamiltonian(n_qubits,J,h)
    h_mat = ham_qu.to_sparse_matrix().toarray()
    D, P = np.linalg.eigh(h_mat)
    exp_h = P @ (np.diag(np.exp(-1j*t*D))@P.conj().T)
    psi_0 = np.zeros(2**n_qubits)
    psi_0[0]=1
    psi_t = exp_h @ psi_0
    mag_op = observable_Z(n_qubits).to_sparse_matrix().toarray()
    mag = np.real(psi_t @ (mag_op @psi_t.conj().T))
    return mag
mag_arr = []
for i in t_arr:
    mag_arr.append(get_M(n_qubits,J,i))
plt.plot(t_arr,mag_arr)

NameError: name 'QubitPauliOperator' is not defined

In [ ]:
big_mag_arr = []
big_t_arr = []
orders = [2]
plt.plot(t_arr,mag_arr,label=f"exact")
for i in orders:
    t_arr, mag_values = time_evolve(n_qubits,t,delta, order=i,h=h,J=J)
    big_mag_arr.append(mag_values)
    big_t_arr.append(t_arr)
    plt.plot(t_arr,mag_values,label=f"order = {i}")

plt.legend()

In [ ]:
plt.plot(np.abs(np.array(mag_values)-np.array(mag_arr)))

In [82]:
for index, row in df.iterrows():
    before_state_vector = []
    after_state_vector = []
    row_array = np.array(row)
    norm = row_array/np.linalg.norm(row_array)
    print(norm)

    x_state = norm
    print(x_state)

    x_state_box = StatePreparationBox(x_state)

    state_circ = Circuit(3)
    state_circ.add_gate(x_state_box,[0,1,2])
    before_state_vector.append(state_circ.get_statevector())
    first_order = prepare_first_order(n_qubits,t,delta,J,h)
    state_circ.append(first_order)
    after_state_vector.append(state_circ.get_statevector())

draw(state_circ)
print(before_state_vector, after_state_vector)

np.allclose(before_state_vector, after_state_vector)

[0.         0.99959608 0.         0.00352502 0.         0.02820019
 0.         0.        ]
[0.         0.99959608 0.         0.00352502 0.         0.02820019
 0.         0.        ]
[0.00000000e+00 9.99496245e-01 9.08137602e-05 5.18935773e-04
 0.00000000e+00 3.17329225e-02 0.00000000e+00 0.00000000e+00]
[0.00000000e+00 9.99496245e-01 9.08137602e-05 5.18935773e-04
 0.00000000e+00 3.17329225e-02 0.00000000e+00 0.00000000e+00]
[8.49920811e-03 5.45891806e-01 6.49042492e-03 8.37384148e-01
 4.64075326e-04 2.19176147e-02 1.65741188e-04 1.39620377e-02]
[8.49920811e-03 5.45891806e-01 6.49042492e-03 8.37384148e-01
 4.64075326e-04 2.19176147e-02 1.65741188e-04 1.39620377e-02]
[5.88325481e-03 9.08092870e-02 2.85290482e-02 9.93765076e-01
 4.42536187e-04 4.99818244e-03 8.87225835e-04 5.75340113e-02]
[5.88325481e-03 9.08092870e-02 2.85290482e-02 9.93765076e-01
 4.42536187e-04 4.99818244e-03 8.87225835e-04 5.75340113e-02]
[1.24483441e-02 4.69854709e-02 1.01195159e-01 9.91946238e-01
 7.25588565e-04 2.5

[array([0.06709532+0.j, 0.65949   +0.j, 0.29891253+0.j, 0.68639913+0.j,
       0.00086757+0.j, 0.00081222+0.j, 0.00354134+0.j, 0.00804609+0.j])] [array([ 0.01746249-0.31070011j,  0.49954721-0.2480074j ,
        0.11305976-0.09105931j,  0.60643445-0.27727752j,
       -0.11685543-0.0074542j , -0.09479332-0.19024675j,
       -0.03264998-0.0452618j , -0.09810649-0.23399926j])]


False